In [8]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv("data/homes_example.csv")
df = df.replace(np.nan, '', regex=True)
df

,STATE,County_name,Comp5star,PROVNUM,PROVNAME,Sum of MDScensus_avg,Sum of Beds_Avail,Count of FACID
0,AK,Anchorage,4,25018,A,10,16,1
1,,,5,25025,B,11,17,1
2,,,,25036,C,12,18,1
3,,Fairbanks North Star,3,25020,D,13,19,1
4,WY,Laramie,1,535013,E,17,28,1
5,,,,535025,F,19,77,1
6,,,5,535032,G,21,14,1
7,,,,535044,H,23,56,1
8,,Natrona,4,535024,K,45,56,1
9,,,,535049,L,93,26,1


In [100]:
state_idxs = df['STATE'].str.findall(r'\w\w')
state_idxs


0     [AK]
1       []
2       []
3       []
4     [WY]
5       []
6       []
7       []
8       []
9       []
10    [VT]
11      []
12      []
13      []
14      []
15      []
16      []
17      []
Name: STATE, dtype: object

In [94]:
s = df['STATE']
l = []
for t in s.items():
    if t[1] != '':
        l.append(t)

l




[(0, 'AK'), (4, 'WY'), (10, 'VT')]